In [1]:
from dataTable import *
from dash import *
import os
A = ExTable('TestData/A.xlsx', 'Sheet1', skipRows=[1])
B = ExTable('TestData/B.xlsx', 'Sheet1', skipRows=[1])
diff = A.Compare(B)
def PDToDash(df, width = '1500px', height = '500px'):
    style_table={
        'overflowX': 'auto',  # 横向滚动
        'overflowY': 'auto',  # 纵向滚动
        'maxHeight': height,  # 限制表格高度
        'maxWidth': width,  # 限制表格宽度
    }
    return dash_table.DataTable(df.to_dict('records'), [{"name": i, "id": i} for i in df.columns], style_table = style_table, )

In [2]:
# Dash 应用
app = Dash()

# 布局
app.layout = html.Div([
    # diffA 和 diffB 并排显示
    html.Div([
        html.Div([
            html.Button('复制A表差异', id='diffA'),
            html.Button('打开A表', id='openA'),
            PDToDash(diff['diffA'], width='750px'),
        ], style={'width': '50%', 'display': 'inline-block', 'verticalAlign': 'top'}),
        html.Div([
            html.Button('打开B表', id='openB'),
            html.Button('复制B表差异', id='diffB'),
            PDToDash(diff['diffB']),
        ], style={'width': '50%', 'display': 'inline-block', 'verticalAlign': 'top'}),
    ], style={'display': 'flex', 'gap': '20px'}),  # 使用 flex 布局并添加间距

    # 其他表格
    html.Button('复制B表增添项', id='add'),
    PDToDash(diff['add']),
    html.Button('复制B表删除项', id='remove'),
    PDToDash(diff['remove']),
])

# 回调函数
@callback(
    Input('diffA', 'n_clicks'),
    Input('openA', 'n_clicks'),
    Input('diffB', 'n_clicks'),
    Input('openB', 'n_clicks'),
    Input('add', 'n_clicks'),
    Input('remove', 'n_clicks'),
    prevent_initial_call=True
)
def OnInteract(a, openA, b, openB, add, remove):
    id = ctx.triggered_id
    if id == 'openA':
        os.system("start " + A.path)
    elif id == 'openB':
        os.system("start " + B.path)
    else:
        diff[id].to_clipboard(excel=True, sep=None, index=False, header=None)

if __name__ == '__main__':
    app.run(debug=False)